# Solving Vehicle Routing Problems (VRPs) using the `spopt.Route` module
*Authors:* [Dylan Skrah](https://github.com/fiendskrah), [Germano Barcelos](https://github.com/gegen07), [Levi J. Wolf](https://github.com/ljwolf)

This notebook will demonstrate how to set up and solve vehicle routing problems using the `spopt.Route` module. This module wraps around the [`routingpy`](https://github.com/mthh/routingpy) library for routing services and the [PyVRP](https://github.com/PyVRP/PyVRP) for specific solve functions. 

## Setting up the OSRM backend service

### Docker container
We suggest using the OSRM backend as a routing service. This module was developed with this backend considered as the default. Because `spopt.Route` invokes the routingpy library, other services supported by routingpy should function as expected, though minor changes to the codebase may be required. See the following table for which services have been confirmed functional.

| Router       | Required Keywords | API key? | Requires Backend? | confirmed functional | 
|--------------|----------|----------| ----------------  | ------------------- |
| OSRM         | base-url | no | yes     | yes | 
| Valhalla | base-url, profile | no | yes | no | 
| HereMaps     |          | Yes      | | no |
| Google       |          | Yes      | | no |
| Graphhopper  |          | Yes      | | no |
| Mapbox OSRM  |          | Yes      | | no |
| OpenRouteService |      | Yes      | | no |
| OpenTripPlanner | | | | N/A (`matrix` not implemented) | 

OSRM exposes their backend for easy requests using a docker image. While this can be a little tedious to set up on your local machine, this allows the `route` module to quickly identify shortest routes between points of interest and solve VRP problems. Find the latest release of the docker image here: [osrm-backend github repository](https://github.com/Project-OSRM/osrm-backend).

### Preliminary data pre-processing
A required preliminary step is to obtain the 'raw' data for the area in which you are operating. This takes the form of `.pbf` files, which can be obtained from the [geofabrik](https://download.geofabrik.de/_) portal. In our case, we obtain the [Ireland and Northern Ireland](https://download.geofabrik.de/europe/ireland-and-northern-ireland.html) `.pbf` file. Once downloaded, the `.pbf` needs to be processed using a series of extraction, partitioning, and customization commands. This is easiest to do as a shell script. The version of this script for this example can be found in this [gist](https://gist.github.com/fiendskrah/f4d267ee7298ff9d0a9feb387b051b39). it looks like this:

```bash
docker run -t -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-extract -p /opt/car.lua /data/ireland-and-northern-ireland-latest.osm.pbf || echo "osrm build failed"
docker run -t -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-partition /data/ireland-and-northern-ireland-latest.osrm || echo "osrm-partition failed"
docker run -t -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-customize /data/ireland-and-northern-ireland-latest.osrm || echo "osrm-customize failed"
echo "osrm server can now be started"
```

This script will create several additional files from the `.pbf` that you downloaded, which are required for identifying shortest routes and solving the VRP using real street network data.

### Activate the backend service
After the pre-processing steps are completed, the OSRM backend docker container can be activated. This container should be 'spun up' as a service, meaning it holds a port on your machine available to hear requests and send those requests to the OSRM servers to obtain routes, distances, and durations. The `spopt.Route` module will then take those data and solve the defined VRP. Once you have the docker image in your directory, start the service by running the following command in your terminal: 

```bash
docker run -t -i -p 5000:5000 -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-routed --algorithm mld --max-table-size 9999999 /data/ireland-and-northern-ireland-latest.osrm
```

Let's breakdown this command. This part:

```bash
docker run -t -i -p 5000:5000 -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-routed
```

says we're connecting to the docker service on the 5000 port. We're porting into the data directory of the backend and starting the osrm-routed service. 

This part:
 
```bash
--algorithm mld --max-table-size 9999999 /data/ireland-and-northern-ireland-latest.osrm
```

says that we're going to use the MLD (Multi-Level Dijkstra) algorithm to identify the most optimal routes in our problem. the `--max-table-size 99999999` argument indicates we are increasing the rate limitations for problem size, which is quite low by default. Finally, `data/ireland-and-northern-ireland-latest.osrm` is where our processed data file is, which tells OSRM where we're trying to operate. 

After running this final command, the service is activated and listening for requests in your terminal. 

## Set up the Vehicle Routing Problem
Our example VRP is a delivery application where all the pubs in Dublin, Ireland need to be supplied their allotment of Guinness barrels.

In [ ]:
import geopandas as gpd
import pandas, numpy, pyvrp, sys

sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # active development; may need to be edited for your local branch. delete after PR is merged.

import spopt
print(spopt.__file__)

In [ ]:
from spopt.route import engine, heuristic, utils

In [ ]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

### Trucks
In the cell below, we define a DataFrame quantifying the available fleet of trucks. Each row represents a different truck type, identified by size (`namesize`) and fuel type (`namefuel`). The `capacity` column indicates how much the truck can carry. `fixed_cost` is the base cost of using the truck, regardless of how far it travels. `cost_per_meter` and `cost_per_minute` represent variable costs that depend on distance and travel time. The `n_truck` column tells us how many of each truck type are available. 

In [ ]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

### Clients and depot(s)
Our example application uses one central depot, but additional depots can be specified. This file contains pubs all across Dublin, stored in a GeoJSON file and read into a GeoDataFrame using GeoPandas. Each row in this table represents a location, either a pub (client) or the Guinness Storehouse (the depot, at index 0), with associated geographic coordinates and attributes relevant to the routing problem.

In [ ]:
gdf = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublinpubs.geojson')

In [ ]:
gdf.shape

In [ ]:
clients = gdf.iloc[1:,:].reset_index(drop=True)
clients = clients.set_index(clients.osmid.astype(str))

In [ ]:
clients.head()

Clients have associated `demand` and `supply` values, which represent how many kegs needs to be delivered to or picked up from that site. The `geometry` column stores the location as a geographic point. Importantly, the gdf is indexed using the `osmid`.

In [ ]:
depot = gdf.iloc[0,:]

We extract the first row as the depot, which serves as the start and end point for all vehicle routes. The remaining rows are identified as clients and indexed by their unique IDs from Openstreetmap.

### Initialize LastMile object
Now we have all the tools to set up and solve a Vehicle Routing Problem. First, we initalize the problem, setting the depot location and optionally setting the operating hours.

In [ ]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)

Then, we add the clients to be serviced in the routing problem:

In [ ]:
print("adding clients")
m.add_clients(
    locations = clients.geometry, 
    delivery = clients.demand,
    pickup = clients.supply,
    time_windows=None,
    service_times=(numpy.log(clients.demand)**2).astype(int)
)

Lastly, we add the available delivery vehicles to the model object.

In [ ]:
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)

## Solve the VRP

All that's left to do is specify the routing engine and initalize the solve. We need to import the associated module from the `routingpy` library:

In [ ]:
from routingpy import OSRM

Finally, call the solve method, specifying the imported module with the `routing` keyword, and pass any required keywords for the engine using hte `routing_kws` dictionary. For OSRM, all that's required is the base_url where the docker container is listening for requests (described above)

In [ ]:
m.solve(stop=pyvrp.stop.MaxRuntime(60), routing=OSRM, routing_kws={"base_url": "http://localhost:5000"})

With the problem solved, we can now write outputs. The `write_result` method produces 3 files:

- `routes.csv`: An overview of routes produced by the solution
- `stops.csv`: Detailed information about each stop along each route
- `map.html`: An html map displaying the solution.

In [ ]:
m.write_result("osrm")

In [ ]:
routes = gpd.read_file('osrm_routes.csv')
stops = gpd.read_file('osrm_stops.csv')

In [ ]:
routes

In [ ]:
stops

Additionally, `spopt.Route` provides support for cases where no routing engine is passed to the solver. In this case, haversine distances are used in place of road data, and the resulting solutions should be interpreted cautiously. 

In [ ]:
m.solve(stop=pyvrp.stop.MaxRuntime(60))

In [ ]:
m.write_result("no-engine")